# IMPORT Dependencies


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import difflib as dl

In [2]:
import pymysql
pymysql.install_as_MySQLdb()

# Create Species Key

In [3]:
# Read csv file into pandas dataframe
csv_file = "Data/Pop.EstByState.csv"
spec_df = pd.read_csv(csv_file)

In [4]:
#Select species columns and drop duplicates
spec_df = spec_df[['Sequence AOS 59', 'English Name', 'Scientific Name']]
spec_df = spec_df.drop_duplicates()
spec_df = spec_df.rename(columns={'Sequence AOS 59': 'SPECIES_ID'})
spec_df['Generic Name'] = spec_df['English Name'].astype(str).str.split().str[1]
# Create key index
species_key = spec_df.set_index(['SPECIES_ID'])
species_key.head()

,English Name,Scientific Name,Generic Name
SPECIES_ID,,,
81,Plain Chachalaca,Ortalis vetula,Chachalaca
94,Mountain Quail,Oreortyx pictus,Quail
99,Northern Bobwhite,Colinus virginianus,Bobwhite
102,Scaled Quail,Callipepla squamata,Quail
104,California Quail,Callipepla californica,Quail


In [5]:
# Read csv file into pandas dataframe
csv_file = "Data/Pop.EstByState.csv"
pop_df = pd.read_csv(csv_file)

In [6]:
#filter for State of Illinois
pop_df = pop_df[(pop_df['Province / State / Territory'] == 'IL')]
# keep neccesary columns


pop_df = pop_df[['Sequence AOS 59','Province / State / Territory', 'Population Estimate',
        'Lower 95% bound', 'Upper 95% bound', 'Estimated % of Global Population',
        'Estimated % of USA/Canada Population', 'Median Estimate', 'Lower 80% bound', 'Upper 80% bound']]
#DROP SPECIES WITH NO POP EST
pop_df = pop_df.dropna(subset = ['Population Estimate'])
# rename columns
pop_df = pop_df.rename(columns={'Sequence AOS 59': 'SPECIES_ID' })

#Set index
bird_populations = pop_df.set_index(['SPECIES_ID'])

In [7]:
#calculate total bird population
bird_populations['Population Estimate']= bird_populations['Population Estimate'].str.replace(',','')
bird_populations['Population Estimate']= pd.to_numeric(bird_populations['Population Estimate'])
tot_pop = bird_populations['Population Estimate'].sum() 
print(tot_pop)

84130646


In [8]:
# Add column for % of state population
bird_populations['Estimated % of State Population'] = (bird_populations['Population Estimate'] / tot_pop)*100
#round to decimals
decimals = 2   
bird_populations['Estimated % of State Population'] = bird_populations['Estimated % of State Population'].apply(lambda x: round(x, decimals))

In [9]:
bird_populations.head(25)

,Province / State / Territory,Population Estimate,Lower 95% bound,Upper 95% bound,Estimated % of Global Population,Estimated % of USA/Canada Population,Median Estimate,Lower 80% bound,Upper 80% bound,Estimated % of State Population
SPECIES_ID,,,,,,,,,,
119,IL,290,0,"1,200",0.0%,0.0%,160,0,820,0.00
123,IL,190000,"110,000","310,000",0.3%,1.2%,"180,000","130,000","270,000",0.23
149,IL,290000,"180,000","450,000",0.2%,1.8%,"280,000","200,000","400,000",0.34
165,IL,120000,"70,000","200,000",0.1%,1.4%,"120,000","80,000","180,000",0.14
198,IL,3200000,"2,100,000","4,800,000",2.1%,2.4%,"3,000,000","2,300,000","4,400,000",3.80
205,IL,140000,"89,000","210,000",1.5%,1.7%,"140,000","100,000","180,000",0.17
209,IL,3300,"1,300","6,600",0.4%,0.4%,"3,000","1,700","5,300",0.00
227,IL,7700,"2,400","17,000",0.0%,0.0%,"6,900","3,700","12,000",0.01
242,IL,30000,"5,900","69,000",1.6%,1.6%,"28,000","12,000","52,000",0.04


# CLEAN UP BIRD STRIKE DATA

In [10]:
# Read csv file into pandas dataframe
csv_file = "Data/IL_airplane_strikes_data_2000-2016.csv"
strike_df = pd.read_csv(csv_file, encoding = 'ISO-8859–1')


In [11]:
#filter for chicago airports and years 2004-2016
strike_df = strike_df[(strike_df['AIRPORT_ID'] == 'KORD') | (strike_df['AIRPORT_ID'] == 'KMDW')]
strike_df = strike_df[(strike_df['INCIDENT_YEAR'] >= 2004) & (strike_df['INCIDENT_YEAR'] <= 2016)]

#keep neccesary columns
bird_strikes = strike_df[['INCIDENT_DATE', 'INCIDENT_MONTH', 'INCIDENT_YEAR', 
         'TIME_OF_DAY', 'TIME','AIRPORT_ID','AIRPORT', 'SPECIES', 'COST_REPAIRS', 
         'EFFECT', 'SKY', 'PRECIP', 'BIRDS_SEEN', 'BIRDS_STRUCK', 'SIZE']]

bird_strikes.head()


,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME_OF_DAY,TIME,AIRPORT_ID,AIRPORT,SPECIES,COST_REPAIRS,EFFECT,SKY,PRECIP,BIRDS_SEEN,BIRDS_STRUCK,SIZE
0,12/29/2016,12,2016,NaN,NaN,KORD,CHICAGO O'HARE INTL ARPT,Snowy owl,NaN,NaN,NaN,NaN,NaN,1,Large
1,12/26/2016,12,2016,Night,2000.0,KORD,CHICAGO O'HARE INTL ARPT,Unknown bird - large,NaN,None,No Cloud,None,1,1,Large
2,12/22/2016,12,2016,NaN,NaN,KMDW,CHICAGO MIDWAY INTL ARPT,Short-eared owl,NaN,NaN,NaN,NaN,NaN,1,Small
3,12/18/2016,12,2016,Day,1701.0,KORD,CHICAGO O'HARE INTL ARPT,Mallard,NaN,NaN,NaN,NaN,NaN,1,Medium
4,12/15/2016,12,2016,NaN,1015.0,KMDW,CHICAGO MIDWAY INTL ARPT,Canada goose,NaN,Precautionary Landing,Some Cloud,None,2 to 10,1,Medium


In [12]:
#attempted to use get close match funtion to do fuzzy match and join species ID
#bird_strikes['SPECIES'] = bird_strikes['SPECIES'].apply(lambda x: dl.get_close_matches(x, species_key['English Name']))



# CLEAN UP BLD STRIKE DATA 

In [13]:
# Read csv file into pandas dataframe
csv_file = "Data/Chicago_bld_strike_data_1978-2016.csv"
bld_strike_df = pd.read_csv(csv_file)

#filter for 2004-2016
bld_strike_df = bld_strike_df[(bld_strike_df['Date'] >= '2004-01-01') & (bld_strike_df['Date'] <= '2016-12-31')]

bld_strike_df.head()

,Genus,Species,Date,Locality
41,Ammodramus,nelsoni,2004-05-18,MP
42,Ammodramus,nelsoni,2004-10-02,MP
43,Ammodramus,nelsoni,2005-09-28,MP
44,Ammodramus,nelsoni,2006-09-20,MP
45,Ammodramus,nelsoni,2007-05-20,MP


# CLEAN UP FLIGHT DATA

In [14]:
# Read csv files into pandas dataframe
csv_file = "Data/MDW Flight Data 2004-2019.csv"
MDW_flights = pd.read_csv(csv_file)
csv_file = "Data/ORD Flight Data 2004-2019.csv"
ORD_flights = pd.read_csv(csv_file)

In [15]:
#append data
flights = MDW_flights.append(ORD_flights, ignore_index=True)

#filter for years 2004-2016
flights = flights[(flights['year'] >= 2004) & (flights['year'] <= 2016)]

#had to correct data file for ' ' in month column
display(flights.columns)

#keep neccessary columns
chicago_flights = flights[['year', 'month', 'airport', 'airport_name', 'arr_flights']]

#rename columns
chicago_flights = chicago_flights.rename(columns={'arr_flights': 'total_flights' })
chicago_flights.head()

Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', ' weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       ' arr_delay', ' carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

,year,month,airport,airport_name,total_flights
0,2004,1,MDW,"Chicago, IL: Chicago Midway International",149.0
1,2004,1,MDW,"Chicago, IL: Chicago Midway International",120.0
2,2004,1,MDW,"Chicago, IL: Chicago Midway International",56.0
3,2004,1,MDW,"Chicago, IL: Chicago Midway International",155.0
4,2004,1,MDW,"Chicago, IL: Chicago Midway International",247.0


# IMPORT INTO MySQL DATABASE

In [16]:
# connect to local database
from config import password

In [24]:
# create connection to database
rds_connection_string = f"root:{password}@127.0.0.1/birds_db"
engine = create_engine(f'mysql://{rds_connection_string}')


In [25]:
#Write DataFrames to MySQL birds_db
chicago_flights.to_sql(name='chicago_flights', con=engine, if_exists='replace', index=False)
bld_strike_df.to_sql(name='bld_strikes', con=engine, if_exists='replace', index=False)
bird_strikes.to_sql(name='bird_strikes', con=engine, if_exists='replace', index=False)
bird_populations.to_sql(name='bird_populations', con=engine, if_exists='replace', index=True)
species_key.to_sql(name='species_key', con=engine, if_exists='replace', index=True)

In [26]:
# check for tables
engine.table_names()

['bird_populations',
 'bird_strikes',
 'bld_strikes',
 'chicago_flights',
 'species_key']

In [19]:
pd.read_sql_query('select * from chicago_flights', con=engine).head()

,year,month,airport,airport_name,total_flights
0,2004,1,MDW,"Chicago, IL: Chicago Midway International",149.0
1,2004,1,MDW,"Chicago, IL: Chicago Midway International",120.0
2,2004,1,MDW,"Chicago, IL: Chicago Midway International",56.0
3,2004,1,MDW,"Chicago, IL: Chicago Midway International",155.0
4,2004,1,MDW,"Chicago, IL: Chicago Midway International",247.0


In [20]:
pd.read_sql_query('select * from bld_strikes', con=engine).head()

,Genus,Species,Date,Locality
0,Ammodramus,nelsoni,2004-05-18,MP
1,Ammodramus,nelsoni,2004-10-02,MP
2,Ammodramus,nelsoni,2005-09-28,MP
3,Ammodramus,nelsoni,2006-09-20,MP
4,Ammodramus,nelsoni,2007-05-20,MP


In [21]:
pd.read_sql_query('select * from bird_strikes', con=engine).head()

,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME_OF_DAY,TIME,AIRPORT_ID,AIRPORT,SPECIES,COST_REPAIRS,EFFECT,SKY,PRECIP,BIRDS_SEEN,BIRDS_STRUCK,SIZE
0,12/29/2016,12,2016,None,NaN,KORD,CHICAGO O'HARE INTL ARPT,Snowy owl,NaN,None,None,None,None,1,Large
1,12/26/2016,12,2016,Night,2000.0,KORD,CHICAGO O'HARE INTL ARPT,Unknown bird - large,NaN,None,No Cloud,None,1,1,Large
2,12/22/2016,12,2016,None,NaN,KMDW,CHICAGO MIDWAY INTL ARPT,Short-eared owl,NaN,None,None,None,None,1,Small
3,12/18/2016,12,2016,Day,1701.0,KORD,CHICAGO O'HARE INTL ARPT,Mallard,NaN,None,None,None,None,1,Medium
4,12/15/2016,12,2016,None,1015.0,KMDW,CHICAGO MIDWAY INTL ARPT,Canada goose,NaN,Precautionary Landing,Some Cloud,None,2 to 10,1,Medium


In [22]:
pd.read_sql_query('select * from bird_populations', con=engine).head()

,SPECIES_ID,Province / State / Territory,Population Estimate,Lower 95% bound,Upper 95% bound,Estimated % of Global Population,Estimated % of USA/Canada Population,Median Estimate,Lower 80% bound,Upper 80% bound,Estimated % of State Population
0,119,IL,290,0,"1,200",0.0%,0.0%,160,0,820,0.00
1,123,IL,190000,"110,000","310,000",0.3%,1.2%,"180,000","130,000","270,000",0.23
2,149,IL,290000,"180,000","450,000",0.2%,1.8%,"280,000","200,000","400,000",0.34
3,165,IL,120000,"70,000","200,000",0.1%,1.4%,"120,000","80,000","180,000",0.14
4,198,IL,3200000,"2,100,000","4,800,000",2.1%,2.4%,"3,000,000","2,300,000","4,400,000",3.80


In [23]:
pd.read_sql_query('select * from species_key', con=engine).head()

,SPECIES_ID,English Name,Scientific Name,Generic Name
0,81,Plain Chachalaca,Ortalis vetula,Chachalaca
1,94,Mountain Quail,Oreortyx pictus,Quail
2,99,Northern Bobwhite,Colinus virginianus,Bobwhite
3,102,Scaled Quail,Callipepla squamata,Quail
4,104,California Quail,Callipepla californica,Quail
